# 청년순이동률 TOP 5

| 지역 |  청년순이동률 |
| -- | -- |
| 전남 무안군 | 4.9 |
| 충남 아산시 | 4.9 |
| 충남 홍성군 | 3.5 |
| 전북 완주군 | 3.5 |
| 경북 안동시 | 2.8 |

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("KOSIS_API_KEY")


In [ ]:
import pandas as pd
import requests as req

base_url = "https://kosis.kr/openapi/statisticsData.do"

params = {
    "method": "getMeta",
    "type": "ITM",
    "apiKey": key,
    "orgId": "101",
    "tblId": "DT_1YL20642",
    "format": "json",
    "jsonVD": "Y",
}

res = req.get(base_url, params=params)

print(res.status_code)

res_text = res.text

print(res_text)

df = pd.DataFrame(res.json())
df.to_csv("../../data/item_code.csv")

In [ ]:
import requests as req

url = "https://kosis.kr/openapi/Param/statisticsParameterData.do"
params = {
    "method": "getList",
    "apiKey": key,
    "orgId": "101",
    "tblId": "DT_1YL20642",
    "itmId": "T001",
    "objL1": "ALL",
    "objL2": "ALL",  # 필수
    "prdSe": "Y",
    "newEstPrdCnt": "3",
    "format": "json",
    "jsonVD": "Y",  # 이거 안넣으면 json 형태로 안옴
}

res = req.get(url, params=params)
print(res.status_code)
print(res.text[:300])


In [ ]:
import pandas as pd

df = pd.DataFrame(res.json())
df.head()

df.info()

df.describe()

In [ ]:
df

In [ ]:
# 1. 데이터 타입 정리
df["DT"] = pd.to_numeric(df["DT"], errors="coerce")
df["PRD_DE"] = df["PRD_DE"].astype(str)

# 2. 제외 지역
exclude_sido = {"11", "23", "31", "21", "22", "24", "25", "26"}  # 수도권+광역시

# 3. 타입 변환
df = df.copy()
df["C1"] = df["C1"].astype(str)

# 4. 시도코드(앞 2자리) 생성
df["SIDO"] = df["C1"].str[:2]

# 5. 청년순이동률만
df = df[df["C2"] == "11"].copy()

# 6. 수도권 + 광역시 제외 (앞2자리 기준)
df = df[~df["SIDO"].isin(exclude_sido)].copy()

# 7. 최신연도 TOP5
latest_year = df["PRD_DE"].astype(str).max()

top_5 = (
    df[df["PRD_DE"].astype(str) == latest_year][["C1", "C1_NM", "SIDO", "DT"]]
    .assign(DT=lambda x: pd.to_numeric(x["DT"], errors="coerce"))
    .dropna(subset=["DT"])
    .rename(columns={"C1_NM": "지역", "DT": "청년순이동률", "SIDO": "시도"})
    .sort_values("청년순이동률", ascending=False)
    .head(5)
)

top_5 = top_5.reset_index()
print(top_5[["C1", "시도", "지역", "청년순이동률"]])

In [ ]:
import json

with open("../../data/SIG.geojson", "r", encoding="utf-8") as f:
    gj = json.load(f)

props = gj["features"][0]["properties"]
print(list(props.keys()))


In [ ]:
CANDIDATES = ["SIG_CD", "sig_cd", "SIGCD", "ADM_CD", "code", "id"]

for k in CANDIDATES:
    if k in props:
        vals = [gj["features"][i]["properties"].get(k) for i in range(5)]
        print(k, vals)


In [ ]:
geo_codes = set(f["properties"]["SIG_CD"] for f in gj["features"])
df_codes = set(top_5["C1"])

print("intersection:", len(geo_codes & df_codes))
print("sample hit:", list(geo_codes & df_codes)[:10])

In [ ]:
# df 코드 샘플 5개
print(top_5["C1"].astype(str).str.zfill(5).tolist())

# GeoJSON 코드 10개 샘플
geo_codes = [f["properties"]["SIG_CD"] for f in gj["features"][:10]]
print(geo_codes)

# df 코드가 GeoJSON에 있는지 직접 조회
geo_set = set(f["properties"]["SIG_CD"] for f in gj["features"])
for c in top_5["C1"].astype(str).str.zfill(5):
    print(c, c in geo_set)


In [ ]:
# GeoJSON에 이름 필드가 뭐가 있는지 확인
print(gj["features"][0]["properties"])


In [ ]:
import folium

m = folium.Map(location=[36.5, 127.9], zoom_start=7, tiles="CartoDB positron")

folium.Choropleth(
    geo_data=gj,
    data=top_5,
    columns=["지역", "청년순이동률"],
    key_on="feature.properties.SIG_KOR_NM",
    fill_color="YlGnBu",
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name="청년 순이동률 (%)",
    nan_fill_color="#e9eef3",
    nan_fill_opacity=1,
).add_to(m)

m


In [ ]:
geo_codes = set(f["properties"]["SIG_CD"] for f in gj["features"])
df_codes = set(top_5["C1"].astype(str))

print(len(geo_codes & df_codes))


In [ ]:
import json
import folium

top_5["C1"] = top_5["C1"].astype(str).str.zfill(5)

with open("SIG.geojson", "r", encoding="utf-8") as f:
    geo = json.load(f)

for f in geo["features"]:
    f["properties"]["SIG_CD"] = str(f["properties"]["SIG_CD"]).zfill(5)


m = folium.Map(location=[36.5, 127.9], zoom_start=7, tiles="CartoDB positron")

folium.Choropleth(
    geo_data=geo,  # ✅ 파일경로 대신 dict
    data=top_5,
    columns=["C1", "청년순이동률"],
    key_on="feature.properties.SIG_CD",
    fill_color="RdYlGn",
    fill_opacity=0.8,
    line_opacity=0.3,
    legend_name="청년 순이동률 (%)",
).add_to(m)

m


In [ ]:
sido_map = {
    "11": "서울",
    "21": "부산",
    "22": "대구",
    "23": "인천",
    "24": "광주",
    "25": "대전",
    "26": "울산",
    "29": "세종",
    "31": "경기",
    "32": "강원",
    "33": "충북",
    "34": "충남",
    "35": "전북",
    "36": "전남",
    "37": "경북",
    "38": "경남",
    "39": "제주",
}

top_5["시도"] = top_5["C1"].astype(str).str[:2].map(sido_map)

top_5 = top_5.drop(columns=["index"])

top_5

In [ ]:
import koreanize_matplotlib  # noqa: F401
import matplotlib.pyplot as plt

top_5["시도명"] = top_5["시도"].map(sido_map).fillna(top_5["시도"])
top_5["라벨"] = top_5["시도명"] + " " + top_5["지역"]

plt.figure()
plt.bar(top_5["라벨"], top_5["청년순이동률"])
plt.title(f"청년순이동률 TOP5 (수도권 + 광역시 제외, {latest_year})")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import folium

# 1) 순이동률 데이터 (예시)
# key 컬럼(지역코드)이 GeoJSON의 feature.properties 안 코드와 동일해야 함
# df = pd.DataFrame({
#     "code": ["11110", "11140", "11200"],   # 예: 시군구 코드
#     "net_mig_rate": [1.2, -0.8, 0.3],      # 순이동률(%) = (순이동/인구)*100
# })

# 2) 지도 중심/줌
m = folium.Map(location=[36.5, 127.9], zoom_start=7, tiles="CartoDB positron")

# 3) Choropleth
folium.Choropleth(
    geo_data="SIG.geojson",  # <-- 시군구 경계 GeoJSON 파일 경로
    data=top_5,
    columns=["C1", "청년순이동률"],
    key_on="feature.properties.SIG_CD",  # <-- GeoJSON 속성키(여기 코드가 있어야 함)
    fill_color="RdYlGn",  # 음수/양수 표현에 좋음(빨강=감소, 초록=증가 느낌)
    fill_opacity=0.75,
    line_opacity=0.2,
    nan_fill_opacity=0.1,
    legend_name="순이동률(%)",
).add_to(m)

m.save("net_migration_rate_map.html")
m


In [ ]:
import os

print(os.getcwd())
print(os.listdir())

In [ ]:
import json

with open("SIG.geojson", encoding="utf-8") as f:
    gj = json.load(f)

print(gj["features"][0]["properties"].keys())
